In [ ]:
import numpy as np
import pandas as pd

In [ ]:
IM_PATH = '../input/ricord-covid19-xray-positive-tests/MIDRC-RICORD/MIDRC-RICORD'

In [ ]:
meta_extradata = pd.read_csv('../input/ricord-covid19-xray-positive-tests/MIDRC-RICORD-meta.csv')
meta_extradata.dropna(inplace = True, subset = ['labels'])

In [ ]:
meta_extradata

In [ ]:
def locate_row_to_delete(a):
    if a.max() <= 0.5 :
        return np.array([np.nan,np.nan,np.nan,np.nan])
    else:
        return a


def encode_labels(df):
    df = df[['fname', 'labels']]
    
    #initialize label columns
    df['Negative for Pneumonia'] = 0
    df['Typical Appearance'] = 0
    df['Indeterminate Appearance'] = 0
    df['Atypical Appearance'] = 0
    
    #Count occurences of each category
    df['Negative for Pneumonia'] = df.labels.apply(lambda x : x.count('Negative'))
    df['Typical Appearance'] = df.labels.apply(lambda x : x.count('Typical'))
    df['Indeterminate Appearance'] = df.labels.apply(lambda x : x.count('Indeterminate'))
    df['Atypical Appearance'] = df.labels.apply(lambda x : x.count('Atypical'))
    
    #df to array for computations
    labels_np = df[['Negative for Pneumonia','Typical Appearance', 'Atypical Appearance','Indeterminate Appearance']].values
    temp = labels_np/labels_np.sum(axis = 1, keepdims=True)
    temp = np.apply_along_axis(locate_row_to_delete, 1, temp) 

    temp -= 0.51
    
    temp[temp > 0] = 1
    temp[temp < 0] = 0
    

    df['Negative for Pneumonia'] = temp[:,0]
    df['Typical Appearance'] = temp[:,1]
    df['Indeterminate Appearance'] = temp[:,2]
    df['Atypical Appearance'] = temp[:,3]
    
    
    df.dropna(subset = ['Negative for Pneumonia'], inplace = True)
    return df

In [ ]:
output = encode_labels(meta_extradata)
output

In [ ]:
output.to_csv('external.csv', index=False)